Name: Lea Wei  
uniqname: zhuoqunw  
Bechdel project visualization  

In [5]:
import altair as alt
import pandas as pd
import numpy as np
#from genderperformr import GenderPerformr
#gp = GenderPerformr()
alt.themes.enable("fivethirtyeight")

ThemeRegistry.enable('fivethirtyeight')

In [6]:
# import data
bech_budget_revenue = pd.read_csv("/Users/zhuoqunwei/Development/UMSI/SI618/project1/cleaned datasets/bech_budget_revenue.csv")
bech_movie = pd.read_csv("/Users/zhuoqunwei/Development/UMSI/SI618/project1/cleaned datasets/bech_movie.csv")
bechdel = pd.read_csv("/Users/zhuoqunwei/Development/UMSI/SI618/project1/cleaned datasets/bechdel.csv")
alt.data_transformers.enable('csv') #deal with extremely large dataset

DataTransformerRegistry.enable('csv')

## Q0: percentage of movies passing the test over decades 

In [7]:
# create year group
bins = [1880, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020]
bechdel['year_group'] = pd.cut(bechdel['year'], bins,
                              labels=['1880-1920', '1920s', '1930s', '1940s','1950s', '1960s', '1970s', '1980s', '1990s','2000s','2010s'])
bechdel.head()

,id,imdbid,rating,title,year,test_result,year_group
0,8040,392728.0,0,Roundhay Garden Scene,1888,Pass 0:Fewer than two women,1880-1920
1,5433,3.0,0,Pauvre Pierrot,1892,Pass 0:Fewer than two women,1880-1920
2,6200,132134.0,0,"Execution of Mary, Queen of Scots, The",1895,Pass 0:Fewer than two women,1880-1920
3,5444,14.0,0,Tables Turned on the Gardener,1895,Pass 0:Fewer than two women,1880-1920
4,5406,131.0,0,Une nuit terrible,1896,Pass 0:Fewer than two women,1880-1920


In [8]:
count_test = bechdel.groupby(["year_group","test_result"])["id"].count()
#count_test["id"]=count_test["id"].fillna(0)
count_test

year_group  test_result                          
1880-1920   Pass 0:Fewer than two women                63
            Pass 1:Women don't talk to each other       2
            Pass 2:Women only talk about men            5
            Pass 3:Passes Bechdel Test                 14
1920s       Pass 0:Fewer than two women                37
            Pass 1:Women don't talk to each other      13
            Pass 2:Women only talk about men           19
            Pass 3:Passes Bechdel Test                 21
1930s       Pass 0:Fewer than two women                25
            Pass 1:Women don't talk to each other      43
            Pass 2:Women only talk about men           45
            Pass 3:Passes Bechdel Test                107
1940s       Pass 0:Fewer than two women                30
            Pass 1:Women don't talk to each other      44
            Pass 2:Women only talk about men           34
            Pass 3:Passes Bechdel Test                104
1950s       Pass 0:Few

In [8]:
percent_test = count_test.groupby(level=[0]).apply(lambda g: g / g.sum()).to_frame(name = 'percent').reset_index()
percent_test.columns = ['Decades', 'Test Result', 'Percent']

percent_test.head()

percent_test.loc[percent_test['Test Result'] == "Pass 0:Fewer than two women", 'Test Result'] = 'Pass 0 of 3'
percent_test.loc[percent_test['Test Result'] == "Pass 1:Women don't talk to each other", 'Test Result'] = 'Pass 1 of 3'
percent_test.loc[percent_test['Test Result'] == "Pass 2:Women only talk about men", 'Test Result'] = 'Pass 2 of 3'
percent_test.loc[percent_test['Test Result'] == "Pass 3:Passes Bechdel Test", 'Test Result'] = 'Pass all 3'


In [9]:
time_trend = alt.Chart(percent_test).mark_bar().encode(
    y = alt.Y("Decades:O", title = ""),
    x = alt.X("Percent", 
              stack="normalize", 
              axis=alt.Axis(format='.0%'),
              title=""),
    color = alt.Color("Test Result:N", legend=alt.Legend(orient='top', title=None)),
    order = "Test Result:N",
    opacity = alt.value(0.9)
)

text = alt.Chart(percent_test).mark_text(dx=-11, dy=3, color='white').encode(
    x=alt.X('Percent', stack="normalize"),
    y=alt.Y('Decades:O'),
    text=alt.Text('Percent', format='.0%'),
    order = "Test Result:N"
)

alt.layer(time_trend, text).properties(height=350, width=650, title={
      "text": ["Percentage of movies passing the bechdel test over decades: since 1880"], 
      "subtitle": ["Criteria:",
                   "1.have at least two named women",
                   "2.who talk to each other",
                   "3.about something besides men",
                   "* combine 1880-1920 because of the lack of data in each decade"],
      "color": "black"
    }).configure_title(fontSize=20, anchor="start").configure_axis(
    grid=False
).configure_view(
    strokeWidth=0
)

alt.LayerChart(...)

# Q1: percentage of movies passing the test by production country (>50)

In [10]:
country_test = bech_movie[["imdbid", "country", "binary_pass"]]
country_test = country_test.drop_duplicates()
country_test

,imdbid,country,binary_pass
0,574,Australia,0
3,2101,USA,0
5,3973,USA,0
7,4972,USA,0
13,6745,USA,1
...,...,...,...
97631,9308382,UK,1
97636,9484998,USA,1
97637,9484998,Hong Kong,1
97642,9683478,USA,1


In [11]:
country_count = country_test.groupby(["country"]).count()["imdbid"].reset_index()
country_count_50 = country_count[country_count["imdbid"]>50]
country_count_50.columns = ["producation country", "number of movies"]
country_count_50.sort_values(by=["number of movies"], ascending=False).reset_index(drop=True)

,producation country,number of movies
0,USA,5332
1,UK,1330
2,France,960
3,Germany,538
4,Canada,533
5,Japan,377
6,Italy,298
7,Spain,238
8,Australia,176
9,China,146


In [12]:
country_pass = alt.Chart(country_test).mark_bar().transform_joinaggregate(
    groupby=["country"],
    percent_pass="mean(binary_pass)",
    movie_count = "count(imdbid)"
).transform_filter(
    alt.datum.movie_count>50
).encode(
    y=alt.Y("country:N", sort="-x", title=""),
    x=alt.X("percent_pass:Q", title=""),
)

text2 = alt.Chart(country_test).mark_text(align='left',baseline='middle',dx=3).transform_joinaggregate(
    groupby=["country"],
    percent_pass="mean(binary_pass)",
    movie_count = "count(imdbid)"
).transform_filter(
    alt.datum.movie_count>50
).encode(
    y=alt.Y("country:N", sort="-x"),
    x=alt.X("percent_pass:Q", axis=alt.Axis(format='.0%')),
    text=alt.Text("percent_pass:Q", format=".2%")
)

country_pass_bar = (country_pass + text2).properties(title={
      "text": ["Percentage of movies passing the bechdel test by production country"],
      "subtitle": ["* Only include countries with more than 50 movies","* Movies with more than one production country have been counted for each country"],
      "color": "black"
    })
(country_pass_bar).configure_title(fontSize=20,
                       anchor="start").configure_axis(
    grid=False
).configure_view(
    strokeWidth=0
)


alt.LayerChart(...)

# Q2: percentage of movies passing the test by genre

In [9]:
genre_test = bech_movie[["imdbid", "genre", "binary_pass"]]
#genre_test = genre_test.drop_duplicates()
#genre_test

In [10]:
genre_num = genre_test.groupby(["genre"]).count()["imdbid"].reset_index()
genre_num.columns = ["genre", "number of movies"]
genre_num.sort_values(by=["number of movies"],ascending=False).reset_index(drop=True)

,genre,number of movies
0,Drama,22284
1,Comedy,12194
2,Action,9810
3,Adventure,8756
4,Romance,6513
5,Thriller,5994
6,Crime,5612
7,Horror,3510
8,Mystery,3473
9,Fantasy,3234


In [11]:
genre_pass = alt.Chart(genre_test).mark_bar().transform_joinaggregate(
    groupby=["genre"],
    percent_pass="mean(binary_pass)"
).encode(
    y=alt.Y("genre:N", sort="-x", title=""),
    x=alt.X("percent_pass:Q", title="")
)

text2 = alt.Chart(genre_test).mark_text(align='left',baseline='middle',dx=3).transform_joinaggregate(
    groupby=["genre"],
    percent_pass="mean(binary_pass)"
).encode(
    y=alt.Y("genre:N", sort="-x"),
    x=alt.X("percent_pass:Q", axis=alt.Axis(format='.0%')),
    text=alt.Text("percent_pass:Q", format=".2%")
)

genre_pass_bar = (genre_pass + text2).properties(title={
      "text": ["Percentage of movies passing the bechdel test by genre"],
      "subtitle": ["* Movies with more than one genre have been counted for each genre"],
      "color": "black"
    })
(genre_pass_bar).configure_title(fontSize=20,
                       anchor="start").configure_axis(
    grid=False
).configure_view(
    strokeWidth=0
)

alt.LayerChart(...)

# Q3: percentage of movies passing the test by IMDb rating

In [ ]:
rating_test = bech_movie[["imdbid", "avg_vote", "total_votes","males_allages_avg_vote", "females_allages_avg_vote", "binary_pass"]]
rating_test = rating_test.drop_duplicates()
rating_test.head()

In [ ]:
bin=[0,4,5,6,7,8,9,10]
rating_test['imdb_star'] = pd.cut(rating_test['avg_vote'], bin,
                              labels=['less than 4⭐️', '4⭐️', '5⭐️','6⭐️', '7⭐️', '8⭐️', '9⭐️'])
rating_test['imdb_star_male'] = pd.cut(rating_test['males_allages_avg_vote'], bin,
                              labels=['less than 4⭐️', '4⭐️', '5⭐️','6⭐️', '7⭐️', '8⭐️', '9⭐️'])

rating_test['imdb_star_female'] = pd.cut(rating_test['females_allages_avg_vote'], bin,
                              labels=['less than 4⭐️', '4⭐️', '5⭐️','6⭐️', '7⭐️', '8⭐️', '9⭐️'])

rating_test.head(10)

In [ ]:
rating_num = rating_test.groupby("imdb_star").count()["imdbid"].reset_index()
rating_num.columns=["imdb rating", "number of vote"]
rating_num

In [ ]:
rating_num_m = rating_test.groupby("imdb_star_male").count()["imdbid"].reset_index()
rating_num_m.columns = ["imdb rating by male", "number of vote"]
rating_num_m

In [ ]:
rating_num_f = rating_test.groupby("imdb_star").count()["imdbid"].reset_index()
rating_num_f.columns = ["imdb rating by male", "number of vote"]
rating_num_f

In [ ]:
rating_pass = alt.Chart(rating_test).mark_bar().transform_joinaggregate(
    groupby=["imdb_star"],
    percent_pass="mean(binary_pass)"
).encode(
    y=alt.Y("imdb_star:N", title="", sort=["less than 4⭐️", 
                                                          "4⭐️",
                                                          "5⭐️",
                                                          "6⭐️",
                                                          "7⭐️",
                                                          "8⭐️"]),
    x=alt.X("percent_pass:Q", title="")
).transform_filter(
    alt.FieldOneOfPredicate(field = 'imdb_star', oneOf = ["less than 4⭐️", 
                                                          "4⭐️",
                                                          "5⭐️",
                                                          "6⭐️",
                                                          "7⭐️",
                                                          "8⭐️"]))


text3 = alt.Chart(rating_test).mark_text(align='left',baseline='middle',dx=3).transform_joinaggregate(
    groupby=["imdb_star"],
    percent_pass="mean(binary_pass)"
).encode(
    y=alt.Y("imdb_star:N", title="", sort=["less than 4⭐️", 
                                                          "4⭐️",
                                                          "5⭐️",
                                                          "6⭐️",
                                                          "7⭐️",
                                                          "8⭐️"]),
    x=alt.X("percent_pass:Q", title="", axis=alt.Axis(format='.0%')),
    text=alt.Text("percent_pass:Q", format=".2%")
).transform_filter(
    alt.FieldOneOfPredicate(field = 'imdb_star', oneOf = ["less than 4⭐️", 
                                                          "4⭐️",
                                                          "5⭐️",
                                                          "6⭐️",
                                                          "7⭐️",
                                                          "8⭐️"]))

rating_pass_bar = (rating_pass + text3).properties(title={
      "text": ["Percentage of movies passing the bechdel test by IMDb rating"],
      "subtitle": ["* Combine 1-3 stars to be 'less than 4 star'","* Movies with more than 9 stars have been excluded due to extremely small amount of data"],
      "color": "black"
    })
(rating_pass_bar).configure_title(fontSize=20,
                       anchor="start").configure_axis(
    grid=False
).configure_view(
    strokeWidth=0
)

In [ ]:
# male
rating_pass_male = alt.Chart(rating_test).mark_bar().transform_joinaggregate(
    groupby=["imdb_star_male"],
    percent_pass="mean(binary_pass)"
).encode(
    y=alt.Y("imdb_star_male:N", title="", sort=["less than 4⭐️", 
                                                          "4⭐️",
                                                          "5⭐️",
                                                          "6⭐️",
                                                          "7⭐️",
                                                          "8⭐️"]),
    x=alt.X("percent_pass:Q", title="")
).transform_filter(
    alt.FieldOneOfPredicate(field = 'imdb_star_male', oneOf = ["less than 4⭐️", 
                                                          "4⭐️",
                                                          "5⭐️",
                                                          "6⭐️",
                                                          "7⭐️",
                                                          "8⭐️"]))


text3_male = alt.Chart(rating_test).mark_text(align='left',baseline='middle',dx=3).transform_joinaggregate(
    groupby=["imdb_star_male"],
    percent_pass="mean(binary_pass)"
).encode(
    y=alt.Y("imdb_star_male:N", title="", sort=["less than 4⭐️", 
                                                          "4⭐️",
                                                          "5⭐️",
                                                          "6⭐️",
                                                          "7⭐️",
                                                          "8⭐️"]),
    x=alt.X("percent_pass:Q", title="", axis=alt.Axis(format='.0%')),
    text=alt.Text("percent_pass:Q", format=".2%")
).transform_filter(
    alt.FieldOneOfPredicate(field = 'imdb_star_male', oneOf = ["less than 4⭐️", 
                                                          "4⭐️",
                                                          "5⭐️",
                                                          "6⭐️",
                                                          "7⭐️",
                                                          "8⭐️"]))

rating_pass_bar_male = (rating_pass_male + text3_male).properties(title={
      "text": ["Male Voter"],
      #"subtitle": ["* Combine 1-3 stars to be 'less than 4 star'","* Movies with more than 9 stars have been excluded due to extremely small amount of data"],
      "color": "black"
    })
(rating_pass_bar_male).configure_title(fontSize=15)

# female
rating_pass_female = alt.Chart(rating_test).mark_bar(color="orange").transform_joinaggregate(
    groupby=["imdb_star_female"],
    percent_pass="mean(binary_pass)"
).encode(
    y=alt.Y("imdb_star_female:N", title="", sort=["less than 4⭐️", 
                                                          "4⭐️",
                                                          "5⭐️",
                                                          "6⭐️",
                                                          "7⭐️",
                                                          "8⭐️"]),
    x=alt.X("percent_pass:Q", title="")
).transform_filter(
    alt.FieldOneOfPredicate(field = 'imdb_star_female', oneOf = ["less than 4⭐️", 
                                                          "4⭐️",
                                                          "5⭐️",
                                                          "6⭐️",
                                                          "7⭐️",
                                                          "8⭐️"]))


text3_female = alt.Chart(rating_test).mark_text(align='left',baseline='middle',dx=3).transform_joinaggregate(
    groupby=["imdb_star_female"],
    percent_pass="mean(binary_pass)"
).encode(
    y=alt.Y("imdb_star_female:N", title="", sort=["less than 4⭐️", 
                                                          "4⭐️",
                                                          "5⭐️",
                                                          "6⭐️",
                                                          "7⭐️",
                                                          "8⭐️"]),
    x=alt.X("percent_pass:Q", title="", axis=alt.Axis(format='.0%')),
    text=alt.Text("percent_pass:Q", format=".2%")
).transform_filter(
    alt.FieldOneOfPredicate(field = 'imdb_star_female', oneOf = ["less than 4⭐️", 
                                                          "4⭐️",
                                                          "5⭐️",
                                                          "6⭐️",
                                                          "7⭐️",
                                                          "8⭐️"]))

rating_pass_bar_female = (rating_pass_female + text3_female).properties(title={
      "text": ["Female voter"],
      #"subtitle": ["* Combine 1-3 stars to be 'less than 4 star'","* Movies with more than 9 stars have been excluded due to extremely small amount of data"],
      "color": "black"
    })
(rating_pass_bar_female).configure_title(fontSize=15)

(rating_pass_bar_male | rating_pass_bar_female).configure_title(fontSize=15).configure_axis(
    grid=False
).configure_view(
    strokeWidth=0
)

# Q4: percentage of movies passing the test by crew gender

In [ ]:
crew_pass = bech_movie[["imdbid", "director", "writer","binary_pass"]]
crew_pass = crew_pass.drop_duplicates()
crew_pass.head()

In [ ]:
new= crew_pass["director"].str.split(" ", n = 1, expand = True)
crew_pass["director_name"]= new[0] 

new2=crew_pass["writer"].str.split(" ", n = 1, expand = True)
crew_pass["writer_name"]= new2[0] 

crew_pass

In [ ]:
labels_d=[]
labels_w=[]
for name in crew_pass["director_name"]:
    #print(name)
    label = gp.predict(name)[1]
    labels_d.append(label)
for name in crew_pass["writer_name"]:
    label=gp.predict(name)[1]
    labels_w.append(label)
crew_pass["director_gender"] = labels_d
crew_pass["writer_gender"] = labels_w

In [ ]:
# director gender & movie count
dir_num = crew_pass.groupby("director_gender").count()["imdbid"].reset_index()
dir_num.columns=["director gender", "number of movies"]
dir_num

In [ ]:
# writer gender & movie count
wr_num = crew_pass.groupby("writer_gender").count()["imdbid"].reset_index()
wr_num.columns=["writer gender", "number of movies"]
wr_num

In [ ]:
# director chart
director_test = alt.Chart(crew_pass).mark_bar().transform_filter(
    alt.FieldOneOfPredicate(field = 'director_gender', oneOf = ["M", "F"])
).transform_joinaggregate(
    groupby=["director_gender"],
    percent_pass="mean(binary_pass)"
).encode(
    y=alt.Y('director_gender:N', sort = ["M","F"], title=""),
    x=alt.X("percent_pass:Q", title="", axis=alt.Axis(format='.0%'))
)

text_d = alt.Chart(crew_pass).mark_text(dx=20).transform_filter(
    alt.FieldOneOfPredicate(field = 'director_gender', oneOf = ["M", "F"])
).transform_joinaggregate(
    groupby=["director_gender"],
    percent_pass="mean(binary_pass)"
).encode(
    y=alt.Y('director_gender:N', sort = ["M","F"], title=""),
    x=alt.X("percent_pass:Q", title=""),
    text=alt.Text("percent_pass:Q", format=".2%")
)

director_chart = (director_test+text_d).properties(title="Director")

In [ ]:
# writer chart
writer_test = alt.Chart(crew_pass).mark_bar(color="orange").transform_filter(
    alt.FieldOneOfPredicate(field = 'writer_gender', oneOf = ["M", "F"])
).transform_joinaggregate(
    groupby=["writer_gender"],
    percent_pass="mean(binary_pass)"
).encode(
    y=alt.Y('writer_gender:N', sort = ["M","F"], title=""),
    x=alt.X("percent_pass:Q", title="", axis=alt.Axis(format='.0%'))
)

text_w = alt.Chart(crew_pass).mark_text(dx=20).transform_filter(
    alt.FieldOneOfPredicate(field = 'writer_gender', oneOf = ["M", "F"])
).transform_joinaggregate(
    groupby=["writer_gender"],
    percent_pass="mean(binary_pass)"
).encode(
    y=alt.Y('writer_gender:N', sort = ["M","F"], title=""),
    x=alt.X("percent_pass:Q", title=""),
    text=alt.Text("percent_pass:Q", format=".2%")
)

writer_chart = (writer_test+text_w).properties(title={
    "text":["Writer"]})

In [ ]:
(director_chart | writer_chart).properties(title={
      "text": ["Percentage of movies passing the bechdel test by crew gender"],
      "subtitle": ["* M:Male","* F:Female", "* Movies with multiple crew members have been counted for each crew member"],
      "color": "black"
    }).configure_title(fontSize=20).configure_axis(
    grid=False
).configure_view(
    strokeWidth=0
)

# Q4: median budget and bechdel test since 1990

In [ ]:
budget_pass = bech_budget_revenue[["adj_budget", "test_result"]]
budget_pass.head()
#len(budget_pass)

In [ ]:
budget_test = alt.Chart(budget_pass).mark_bar().transform_filter(
    alt.datum.adj_budget != None
).transform_joinaggregate(
    groupby=["test_result"],
    median_budget="median(adj_budget)"
).encode(
    x=alt.X("median_budget:Q", title=""),
    y=alt.Y("test_result", sort="x", title=""),
    color=alt.condition(
    alt.datum.test_result == "Pass 3:Passes Bechdel Test",  
    alt.value('orange'),     
    alt.value('steelblue')   
) 
)

text_budget = alt.Chart(budget_pass).mark_text(dx=25).transform_filter(
    alt.datum.adj_budget != None
).transform_joinaggregate(
    groupby=["test_result"],
    median_budget="median(adj_budget)"
).encode(
    x=alt.X("median_budget:Q"),
    y=alt.Y("test_result", sort="x"),
    text=alt.Text("median_budget:Q", format="$.4s")
)

(budget_test+text_budget).properties(
    title={
        "text":["Median budget and Bechdel test (since 1990)"],
        "subtitle":["2019 dollar"]
    }
).configure_title(anchor="start",fontSize=20).configure_axis(
    grid=False
).configure_view(
    strokeWidth=0
)

# Q5: median box office and bechdel test since 1990

In [ ]:
#bech_budget_revenue.head()
#len(bech_budget_revenue)

In [ ]:
dom_box_pass = bech_budget_revenue[["adj_domgross", "test_result"]]
dom_box_pass.head()
# dom_box_pass["adj_domgross"].count() 2118 non-empty adj_domgross

In [ ]:
# domestic
dom_test = alt.Chart(dom_box_pass).mark_bar().transform_filter(
    alt.datum.adj_domgross != None
).transform_joinaggregate(
    groupby=["test_result"],
    median_revenue="median(adj_domgross)"
).encode(
    x=alt.X("median_revenue:Q", title=""),
    y=alt.Y("test_result", title=""),
    color=alt.condition(
    alt.datum.test_result == "Pass 3:Passes Bechdel Test",  
    alt.value('orange'),     
    alt.value('steelblue')   
)
)

text_budget = alt.Chart(dom_box_pass).mark_text(dx=25).transform_filter(
    alt.datum.adj_domgross != None
).transform_joinaggregate(
    groupby=["test_result"],
    median_revenue="median(adj_domgross)"
).encode(
    x=alt.X("median_revenue:Q"),
    y=alt.Y("test_result"),
    text=alt.Text("median_revenue:Q", format="$.4s")
)

domestic = (dom_test+text_budget).properties(
    title={
        "text":["Domestic"]
    }
)
#domestic

In [ ]:
int_box_pass = bech_budget_revenue[["adj_intgross", "test_result"]]
int_box_pass.head()
#int_box_pass["adj_intgross"].count() 2015 non-empty adj_intgross

In [ ]:
# international
int_test = alt.Chart(int_box_pass).mark_bar().transform_filter(
    alt.datum.adj_intgross != None
).transform_joinaggregate(
    groupby=["test_result"],
    median_revenue="median(adj_intgross)"
).encode(
    x=alt.X("median_revenue:Q", title=""),
    y=alt.Y("test_result", title=""),
    color=alt.condition(
    alt.datum.test_result == "Pass 3:Passes Bechdel Test",  
    alt.value('orange'),     
    alt.value('steelblue')   
))

text_budget_int = alt.Chart(int_box_pass).mark_text(dx=25).transform_filter(
    alt.datum.adj_intgross != None
).transform_joinaggregate(
    groupby=["test_result"],
    median_revenue="median(adj_intgross)"
).encode(
    x=alt.X("median_revenue:Q"),
    y=alt.Y("test_result"),
    text=alt.Text("median_revenue:Q", format="$.4s")
)

international = (int_test+text_budget_int).properties(
    title={
        "text":["International"]
    }
)

#international

In [ ]:
(domestic|international).properties(
    title={
        "text": ["Median box office and Bechdel Test (since 1990)"],
        "subtitle": ["2019 dollar"]
    }
).configure_title(fontSize=20).configure_axis(
    grid=False
).configure_view(
    strokeWidth=0
)

In [ ]:
# ROI
dom_roi_pass = bech_budget_revenue[["dom_roi", "test_result"]].dropna()
#dom_roi_pass.head()
len(dom_roi_pass)
# dom_roi_pass["dom_roi"].count() 2101 non-empty

In [ ]:
# domestic roi
dom_roi_test = alt.Chart(dom_roi_pass).mark_bar().transform_joinaggregate(
    groupby=["test_result"],
    median_roi="median(dom_roi)"
).encode(
    x=alt.X("median_roi:Q", title=""),
    y=alt.Y("test_result", title=""),
    color=alt.condition(
    alt.datum.test_result == "Pass 3:Passes Bechdel Test",  
    alt.value('orange'),     
    alt.value('steelblue')   )
)

text_domroi = alt.Chart(dom_roi_pass).mark_text(dx=15, color="black").transform_joinaggregate(
    groupby=["test_result"],
    median_roi="median(dom_roi)"
).encode(
    x=alt.X("median_roi:Q", title=""),
    y=alt.Y("test_result", title=""),
    text=alt.Text("median_roi:Q", format="$.2f")
)

dom_roi = (dom_roi_test+text_domroi).properties(
    title={
        "text":["Domestic"]
    }
)

#dom_roi

In [ ]:
int_roi_pass = bech_budget_revenue[["int_roi", "test_result"]].dropna()
len(int_roi_pass)
#int_roi_pass.head()

In [ ]:
# international roi
int_roi_test = alt.Chart(int_roi_pass).mark_bar().transform_joinaggregate(
    groupby=["test_result"],
    median_roi="median(int_roi)"
).encode(
    x=alt.X("median_roi:Q", title=""),
    y=alt.Y("test_result", title=""),
    color=alt.condition(
    alt.datum.test_result == "Pass 3:Passes Bechdel Test",  
    alt.value('orange'),     
    alt.value('steelblue')   )
)

text_introi = alt.Chart(int_roi_pass).mark_text(dx=15, color="black").transform_joinaggregate(
    groupby=["test_result"],
    median_roi="median(int_roi)"
).encode(
    x=alt.X("median_roi:Q", title=""),
    y=alt.Y("test_result", title=""),
    text=alt.Text("median_roi:Q", format="$.2f")
)

int_roi = (int_roi_test+text_introi).properties(
    title={
        "text":["International"]
    }
)

#int_roi

In [ ]:
(dom_roi | int_roi).properties(
    title={
        "text": ["Median ROI and Bechdel Test (since 1990)"],
        "subtitle": ["2019 dollar"]
    }
).configure_title(fontSize=20).configure_axis(
    grid=False
).configure_view(
    strokeWidth=0
)